# Here we compare the Results of ChatGPT against the manuell Results

In [71]:
import pandas as pd
import os, json
from support_functions.load_data import load_datasets_with_annotations as loading

### Loading the data from excel data

In [72]:
def load_datasets_add_line() -> dict[str, list]:
    datasets: dict[str, list] = loading()
    for key in datasets.keys():
        current_dataset = datasets[key]
        for i, item in enumerate(current_dataset, start=1):
            item = {'counter': i, **item}
            current_dataset[i-1] = item
    return datasets

In [73]:
def prepaire_excel_data() -> pd:
    current_directory = os.getcwd()
    suffix = '\\redundanciesbyannotations'
    directory_excel = current_directory[:-len(suffix)] if current_directory.endswith(suffix) else current_directory
    directory_excel += "\\Dataset\\Evaluation_v4.xlsx"
    excel_data = pd.read_excel(directory_excel, usecols=lambda column: 'Item' not in column, skiprows=range(1), sheet_name="Ground Truth")
    excel_data = excel_data.drop(columns=['Unnamed: 0', 'Unnamed: 12', 'total', 'main', 'benefit', 'total.1', 'main.1', 'benefit.1'], axis=1)
    excel_data = excel_data.rename(columns={'Main Part \nPartial': 'Main Part Partial', 'Main Part \nFull': 'Main Part Full', 'Benefit\nPartial':'Benefit Partial','Benefit\nFull':'Benefit Full'})
    excel_data.index += 1
    excel_data = excel_data.fillna("Empty")
    
    ## Map line number to user story number
    excel_data.insert(loc=4, column='Corresponding USID 1', value=0)
    excel_data.insert(loc=5, column='Corresponding USID 2', value=0)
    datasets = load_datasets_add_line()
    for idx in range(len(excel_data)):
        redundant_pairs = excel_data.iat[idx, 1]
        parts = redundant_pairs.split('_')
        first_number = int(parts[2])
        second_number = int(parts[-1])
        project_number = f"#{excel_data.iat[idx, 0]}#".upper()
        project_data = datasets[project_number]
        for item in project_data: 
            if item['counter'] == first_number:
                excel_data.iat[idx, 4] = item['id']
            if item['counter'] == second_number:
                excel_data.iat[idx, 5] = item['id']
    return excel_data

In [74]:
excel_data = prepaire_excel_data()
excel_data.head(3)

,Project Number,Redundant Pairs,User Stories Texts,Total Redundancy Clause *,Corresponding USID 1,Corresponding USID 2,Main Part Redundancy Clause**,Benefit Part Redundancy Clause ***,Main Part Partial,Main Part Full,Benefit Partial,Benefit Full
1,G03,user_story_12_AND_user_story_39,"user_story_12: #g03# as a staff member, i want...",1,326,353,0,1,False,False,True,False
2,G03,user_story_17_AND_user_story_30,"user_story_17: #g03# as a staff member, i want...",1,331,344,0,1,False,False,True,False
3,G04,user_story_05_AND_user_story_12,"user_story_05: #g04# as a user, i want to have...",1,168,175,0,1,False,False,False,True


### Loading ChatGPT results

In [83]:
directories_to_ignore:dict = ["annotations", "dependency_graph"]
def load_chat_gpt_results() -> list[dict]:
    directory_results = os.path.join(os.getcwd(), "results")
    category_data: dict = {}
    for root, dirs, files in os.walk(directory_results):
        dirs[:] = [d for d in dirs if d not in directories_to_ignore]
        temp_category_data: dict = {}
        for filename in files:
            file_path = os.path.join(root, filename)
            if filename.endswith('.json'):
                with open(file_path, 'r') as file:
                    data = json.load(file)
                    temp_category_data[filename] = data
        if (temp_category_data):
            print(root.split("\\")[-1])
            category_data[root.split("\\")[-1]] = temp_category_data
    return category_data

In [82]:
chat_gpt_results: dict = load_chat_gpt_results()

redundancy-model-gpt-3.5-turbo
redundancy-model-gpt-4-turbo


### Analyse - TP, FP, TN, FN, Recall etc, confusion matrix 